In [3]:
import operator
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish, Tool
from langchain_core.messages import BaseMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt.tool_executor import ToolExecutor
from langchain import hub

# 기존 상태 클래스 정의
class AgentState(TypedDict):
    query: str
    context: Annotated[List[str], operator.add]
    intermediate_steps: Annotated[List[Union[AgentAction, AgentFinish]], operator.add]
    answer: str

# 1. 환경 설정 및 데이터 로딩 (기존 코드와 동일)
from dotenv import load_dotenv
load_dotenv()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
loader = Docx2txtLoader('./tax.docx')
temp = loader.load_and_split(text_splitter=text_splitter)
document_list = temp[0:80]

embedding = OpenAIEmbeddings(model='text-embedding-3-large')
vector_store = Chroma(
    collection_name='chroma-tax',
    embedding_function=embedding,
    persist_directory='./chroma-tax'
)
retriever = vector_store.as_retriever(search_kwargs={'k': 3})

llm = ChatOpenAI(model="gpt-4o")

# 2. Tool 정의
# 기존 함수들을 LangChain Tool로 감싸서 'tools' 노드에서 실행 가능하도록 만듭니다.
# Tool은 특정 작업을 수행하는 데 사용되는 함수나 클래스입니다.

def retrieve_tool(query: str) -> List[str]:
    """사용자의 질문에 기반하여 벡터 스토어에서 관련 문서를 검색합니다."""
    docs = retriever.invoke(query)
    return [doc.page_content for doc in docs]

def rewrite_tool(query: str) -> str:
    """사용자의 질문을 사전에 고려하여 변경합니다."""
    dictionary = "사람과 관련된 표현 -> 거주자"
    rewrite_prompt = PromptTemplate.from_template(
        f"""사용자의 질문을 보고, 우리 사전을 참고해서 사용자의 질문을 변경해 주세요.
        사전: {dictionary}
        질문: {{query}}
        """
    )
    rewrite_chain = rewrite_prompt | llm | StrOutputParser()
    response = rewrite_chain.invoke({'query': query})
    return response

# LangChain Tools 리스트
tools = [
    Tool(
        name="retrieve_documents",
        func=retrieve_tool,
        description="질문에 대한 답변을 생성하기 위해 관련 문서를 검색합니다."
    ),
    Tool(
        name="rewrite_query",
        func=rewrite_tool,
        description="질문이 검색에 부적합할 때, 사전을 참고하여 질문을 재작성합니다."
    )
]

tool_executor = ToolExecutor(tools)

# 3. Agent 노드 정의
# Agent는 LLM을 사용하여 어떤 Tool을 호출할지 결정하고, 최종 답변을 생성합니다.

agent_prompt = hub.pull("hwchase17/react")
agent_prompt = agent_prompt.partial(tools=", ".join([tool.name for tool in tools]), tool_names=", ".join([tool.name for tool in tools]))

agent_chain = agent_prompt | llm | StrOutputParser()

def run_agent(state: AgentState) -> AgentState:
    agent_response = agent_chain.invoke(state)
    return {"intermediate_steps": [AgentAction(tool='tool_name_placeholder', tool_input=agent_response, log=agent_response)]}

# 4. Graph 구성
# AgentState를 사용하여 상태를 관리하고, 'agent'와 'tools' 노드를 연결합니다.

builder = StateGraph(AgentState)
builder.add_node("agent", run_agent)
builder.add_node("tools", tool_executor)

builder.add_edge(START, "agent")
builder.add_edge("tools", "agent")

def route_to_tools(state: AgentState):
    # Agent의 응답을 기반으로 다음 노드를 결정하는 로직
    # 여기서는 Agent의 응답을 파싱하여 Tool 호출 여부를 결정합니다.
    # 이 부분은 LangGraph의 Agentic Loop 패턴을 따릅니다.
    # Agent가 Tool을 호출하도록 응답하면 'tools'로, 아니면 'end'로 이동합니다.
    if "FINAL ANSWER" in state['intermediate_steps'][-1].log:
        return END
    return "tools"

builder.add_conditional_edges("agent", route_to_tools)

graph = builder.compile()

# 5. 실행
# 이제 Agent 노드를 시작으로 그래프를 실행합니다.
initial_state = {"query": "연봉 5천만원 직장인의 소득세는?"}
response = graph.invoke(initial_state)

print(response['answer'])

ImportError: cannot import name 'Tool' from 'langchain_core.agents' (c:\workspace\uv_work\.venv\Lib\site-packages\langchain_core\agents.py)